In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv('dataset3.csv')

print(df.head(), df.shape)

   age  sex     bmi  children  smoker     region      charges  \
0   19    1  27.900         0       1  southwest  16884.92400   
1   18    0  33.770         1       0  southeast   1725.55230   
2   28    0  33.000         3       0  southeast   4449.46200   
3   33    0  22.705         0       0  northwest  21984.47061   
4   32    0  28.880         0       0  northwest   3866.85520   

   bmi_smoker_interaction  
0                    27.9  
1                     0.0  
2                     0.0  
3                     0.0  
4                     0.0   (1337, 8)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer
from sklearn.model_selection import GridSearchCV
import math

# separation des features et de la variable cible
X = df.drop('charges', axis=1)
y = df[['charges']]
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

def log_transform(x):
    return np.log(x + 1)

log_transformer = FunctionTransformer(log_transform)

preprocessor_num = Pipeline(steps=[
    ('log', log_transformer),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())
])
preprocessor_cat = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
])

# preprocessing avec transformation des categories et des nums
preprocessor = ColumnTransformer(
    transformers=[
        ('num', preprocessor_num , ['age', 'bmi', 'children', 'bmi_smoker_interaction']),
        ('cat', preprocessor_cat , ['region','sex', 'smoker']),
    ]
)


verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 7)
      Y (la variable cible) : (1337, 1)
 verif du split 80 20 
80% du dataset : X train -> (1136, 7), Y train -> (1136, 1)
20% du dataset : X test -> (201, 7), Y test -> (201, 1)


In [17]:
from sklearn.dummy import DummyRegressor

# création du pipeline dummy
pipeline_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DummyRegressor(strategy='mean'))])


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement puis predire
# sur l'ensemble de test avec le meme pipeline
pipeline_dummy.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_dummy.predict(X_test)
#print(y_pred_dummy)

#test avec les differentes métriques pour le modele dummy : 
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle dummy: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle dummy: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle dummy: {rmse}')

Mean Squared Error du modèle dummy: 145320521.96584958
Coefficient de determination R² du modèle dummy: -0.0027928862543942223
Root Mean Squared Error (RMSE) du modèle dummy: 12054.89618229247


In [18]:
# Modele de regression lineaire

# création du pipeline de prétraitement et du modèle linear regression
pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', LinearRegression())])


# entraîner le pipeline sur les données d'entraînement 
pipeline_lr.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_lr.predict(X_test)
print(y_pred)

#test des les differents metriques sur modele linear regression
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle linear regression: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle linear regression: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle linear regression: {rmse}')

[[ 3061.88082114]
 [35171.76570092]
 [ 6871.41271491]
 [40277.57715372]
 [29215.22763672]
 [ 7054.31177625]
 [ 3669.73243318]
 [ 4711.61465566]
 [ 3434.92670354]
 [39500.12438196]
 [10553.84948957]
 [12827.58077833]
 [ 3669.01432013]
 [13089.02771045]
 [ 2896.28232717]
 [ 2612.98673315]
 [ 2347.86566414]
 [ 7029.61287321]
 [ 5210.47290723]
 [21071.00381976]
 [14783.36320812]
 [10459.96062719]
 [13949.54325772]
 [14638.73575417]
 [ 6273.72223521]
 [ 4862.956899  ]
 [12740.05257638]
 [11578.73486973]
 [ 3873.97423134]
 [ 2788.56730734]
 [ 4832.4118377 ]
 [13587.08947774]
 [11342.31914579]
 [20112.67247275]
 [29778.1139023 ]
 [49928.57839263]
 [ 5042.20702138]
 [13385.29912607]
 [16561.85638887]
 [11318.12899734]
 [ 6366.49040257]
 [ 6162.22586074]
 [26837.16324232]
 [14209.13942721]
 [31065.12054066]
 [ 3803.97129406]
 [11443.2502375 ]
 [12070.37939238]
 [43716.03059577]
 [11719.43753397]
 [ 5842.99842183]
 [41497.29836002]
 [15829.41031531]
 [14559.39949532]
 [ 3245.97287169]
 [15285.10

In [19]:
# Calculez le RMSE (la racine carrée du MSE) pour obtenir une idée de l'erreur en termes des unités d'origine de vos données. 
# Comparez le RMSE à la moyenne des 'charges' (df['charges'].mean()) pour évaluer l'erreur relative.

# RMSE plus élevé que la moyenne : le modèle a du mal à capturer la variation et les tendances dans les données. 
# Il prédit généralement des valeurs qui sont relativement éloignées des valeurs réelles.

# Erreur systématique :  le modèle a tendance à sous-estimer ou surestimer systématiquement les valeurs cibles.

# Modèle insuffisamment performant : Un RMSE élevé peut également indiquer que le modèle que vous utilisez n'est pas suffisamment complexe 
# pour capturer les relations présentes dans les données. Cela peut signifier que le modèle ne tient pas compte de toutes les caractéristiques
# pertinentes ou que la stratégie de prédiction choisie (comme la moyenne) n'est pas adaptée aux données.

# En général, un bon modèle de régression devrait avoir un RMSE inférieur à la moyenne de la variable cible, 
# car cela indiquerait que le modèle est capable de faire des prédictions qui se rapprochent des valeurs réelles en moyenne. 
# L'objectif est de minimiser le RMSE pour obtenir des prédictions précises.

print(df['charges'].mean())
print(rmse)

#donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy 
#en général, le modèle dummy est un modèle très simple et naïf qui est utilisé comme baseline 
#pour évaluer la performance d'autres modèles plus sophistiqués.

13279.121486655948
3791.569372769152


donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy <br>
est un modèle très simple et naïf qui est utilisé comme baseline pour évaluer la performance d'autres modèles plus sophistiqués

In [20]:
# AVEC LE Modele de regression LASSO

# création du pipeline de prétraitement et du modèle LASSO
pipeline_lasso = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', Lasso())])

#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': [0.1, 1, 10, 100]  
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle lasso: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle lasso: {rmse} \n rappel moyenne charge : 13279')



/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.046e+10, tolerance: 1.315e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.142e+10, tolerance: 1.298e+07
  model = cd_fast.enet_coordinate_descent(


/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.162e+10, tolerance: 1.397e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.140e+10, tolerance: 1.381e+07
  model = cd_fast.enet_coordinate_descent(
/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

Mean Squared Error du modèle lasso: 13963530.555437753
Coefficient de determination R² du modèle lasso: 0.9036438286997107
Root Mean Squared Error (RMSE) du modèle lasso: 3736.7807743347416 
 rappel moyenne charge : 13279


/home/utilisateur/Bureau/ml/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.218e+10, tolerance: 1.667e+07
  model = cd_fast.enet_coordinate_descent(


In [21]:
# Modele de regression RIDGE

# création du pipeline de prétraitement et du modèle RIDGE
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', Ridge())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_ridge = {
    'regression__alpha': [0.01, 0.1, 1, 10, 100]  
}

### grid search
grid_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_ridge.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_ridge.predict(X_test)
# print(y_pred)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print(f'Coefficient de determination R² du modèle ridge: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle ridge: {rmse} \n rappel moyenne charge : 13279')



Coefficient de determination R² du modèle ridge: 0.9026759424286501
Root Mean Squared Error (RMSE) du modèle ridge: 3755.501638627474 
 rappel moyenne charge : 13279
